# Set environment variables

In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
# Apply a patch to allow nested asyncio loops in Jupyter
import nest_asyncio
nest_asyncio.apply()

# Langgraph

## Install dependencies

In [2]:
! pip install -U langchain_community langchain-openai langchain-anthropic langchain langgraph bs4 langchain_core


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


## Load Docs

In [3]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/how_to/sequence/#related"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

## Create CodeChain

In [63]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

### OpenAI

# Grader prompt
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is a full set of LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user
    question based on the above provided documentation. Ensure any code you provide can be executed \n
    with all required imports and variables defined. Structure your answer with a description of the code solution. \n
    Then list the imports. And finally list the functioning code block. Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# Data model
class code(BaseModel):
    """Schema for code solutions to questions about LCEL."""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")


expt_llm = "gpt-4o"
llm = ChatOpenAI(temperature=0, model=expt_llm)
code_gen_chain_oai = code_gen_prompt | llm.with_structured_output(code)
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain_oai.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
solution

code(prefix="To build a Retrieval Augmented Generation (RAG) chain in LCEL, you need to combine a retriever with a language model to generate responses based on retrieved documents. The process involves creating a retriever to fetch relevant documents, a prompt template to format the input for the language model, and then chaining these components together. The retriever fetches documents based on a query, and the language model generates a response using the retrieved documents as context. Here's how you can set up a simple RAG chain using LangChain.", imports='from langchain_core.retrievers import SimpleRetriever\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_openai import ChatOpenAI\nfrom langchain_core.output_parsers import StrOutputParser', code='# Initialize the retriever\nretriever = SimpleRetriever()\n\n# Define a prompt template that includes the retrieved documents\nprompt = ChatPromptTemplate.from_template("Using the following documents: {documents}, 

In [69]:
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

### Anthropic

# Prompt to enforce tool use
code_gen_prompt_claude = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions> You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user  question based on the \n
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined. Structure your answer: 1) a prefix describing the code solution, 2) the imports, 3) the functioning code block. \n
    Invoke the code tool to structure the output correctly. </instructions> \n Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)


# LLM
expt_llm = "claude-3-opus-20240229"
llm = ChatAnthropic(
    model=expt_llm,
    default_headers={"anthropic-beta": "tools-2024-04-04"},
)

structured_llm_claude = llm.with_structured_output(code, include_raw=True)


# Optional: Check for errors in case tool use is flaky
def check_claude_output(tool_output):
    """Check for parse error or failure to call the tool"""

    # Error with parsing
    if tool_output["parsing_error"]:
        # Report back output and parsing errors
        print("Parsing error!")
        raw_output = str(tool_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"Error parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        )

    # Tool was not invoked
    elif not tool_output["parsed"]:
        print("Failed to invoke tool!")
        raise ValueError(
            "You did not use the provided tool! Be sure to invoke the tool to structure the output."
        )
    return tool_output


# Chain with output check
code_chain_claude_raw = (
    code_gen_prompt_claude | structured_llm_claude | check_claude_output
)


def insert_errors(inputs):
    """Insert errors for tool parsing in the messages"""

    # Get errors
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "assistant",
            f"Retry. You are required to fix the parsing errors: {error} \n\n You must invoke the provided tool.",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }


# This will be run as a fallback chain
fallback_chain = insert_errors | code_chain_claude_raw
N = 3  # Max re-tries
code_gen_chain_re_try = code_chain_claude_raw.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)


def parse_output(solution):
    """When we add 'include_raw=True' to structured output,
    it will return a dict w 'raw', 'parsed', 'parsing_error'."""

    return solution["parsed"]


# Optional: With re-try to correct for failure to invoke tool
code_gen_chain = code_gen_chain_re_try | parse_output

# No re-try
code_gen_chain = code_gen_prompt_claude | structured_llm_claude | parse_output

code_gen_chain = code_gen_chain_oai

In [70]:
# Test
question = "How do I build a RAG chain in LCEL?"
solution = code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)
solution

code(prefix="To build a Retrieval Augmented Generation (RAG) chain in LCEL, you need to combine a retriever with a language model to generate responses based on retrieved documents. The process involves setting up a retriever to fetch relevant documents and then using a language model to generate a response based on those documents. Here's a step-by-step guide to building a RAG chain:", imports='from langchain_core.retrievers import VectorStoreRetriever\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_openai import ChatOpenAI\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.runnables import RunnableParallel', code='# Step 1: Set up the retriever\nretriever = VectorStoreRetriever(vector_store=my_vector_store, search_type=\'similarity\', k=5)\n\n# Step 2: Define the prompt template\nprompt_template = ChatPromptTemplate.from_template("Based on the following documents, answer the question: {question}\\nDocuments: {documents}")\n\n# Step 

## Create Langgraph State

In [71]:
from typing import List
from typing_extensions import TypedDict


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
    """

    error: str
    messages: List
    generation: str
    iterations: int

## Create Langgraph Graph

In [72]:
### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = "do not reflect"

### Nodes


def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection

    # Add reflection
    reflections = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


### Edges


def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

# Create Langgraph Nodes Mapping

In [73]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation solution
workflow.add_node("check_code", code_check)  # check code
workflow.add_node("reflect", reflect)  # reflect

# Build graph
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "check_code")
workflow.add_conditional_edges(
    "check_code",
    decide_to_finish,
    {
        "end": END,
        "reflect": "reflect",
        "generate": "generate",
    },
)
workflow.add_edge("reflect", "generate")
app = workflow.compile()

In [74]:
question = "How can I directly pass a string to a runnable and use it to construct the input needed for my prompt?"
solution = app.invoke({"messages": [("user", question)], "iterations": 0, "error": ""})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
Why do bears have sticky fur?

Because they always use bear-ly any shampoo!
---NO CODE TEST FAILURES---
---DECISION: FINISH---


In [75]:
solution["generation"]

code(prefix="To directly pass a string to a runnable and use it to construct the input needed for your prompt, you can use a lambda function to transform the string into the required input format. In this example, we'll pass a string representing a topic to a lambda function, which will convert it into a dictionary format expected by the prompt template. This dictionary is then passed through the chain of runnables, which includes a prompt template, a chat model, and an output parser.", imports='from langchain_openai import ChatOpenAI\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.prompts import ChatPromptTemplate', code='# Initialize the chat model\nmodel = ChatOpenAI(model="gpt-4o-mini")\n\n# Create a prompt template\nprompt = ChatPromptTemplate.from_template("tell me a joke about {topic}")\n\n# Chain the prompt, model, and output parser together\nchain = prompt | model | StrOutputParser()\n\n# Use a lambda function to convert a string into the requir

# CrewAI

## Creating Crew

In [82]:
# Importing Crew related components
from crewai import Agent, Task, Crew

# Importing CrewAI Tools
from crewai_tools import WebsiteSearchTool

# Importing Pydantic
from pydantic import BaseModel, Field

class CodeSolution(BaseModel):
  prefix: str = Field(description="Description of the problem and approach")
  imports: str = Field(description="Code block import statements")
  code: str = Field(description="Code block not including import statements")

# Create the coding assistant agent
coding_assistant = Agent(
    role='Coding Assistant',
    goal='Provide accurate and executable code solutions using LCEL',
    backstory="""You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n
    Answer the user  question based on the \n
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined.""",
    verbose=False,
    llm='gpt-4o'
)

# Create task for code generation
code_generation_task = Task(
    description="""Answer the user question based on the above provided documentation. Ensure any code you provide can be executed
    with all required imports and variables defined. Structure your answer:
    1) a prefix describing the code solution
    2) the imports
    3) the functioning code block

    Your coding task:
    {question}
    """,
    expected_output="Code solution with prefix description, imports, and executable code block",
    agent=coding_assistant,
    output_pydantic=CodeSolution
)

# Create the crew
code_crew = Crew(
    agents=[coding_assistant],
    tasks=[code_generation_task],
    verbose=False
)

2024-12-16 11:17:32,412 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


In [83]:
# code_crew.train(
#   n_iterations=2,
#   filename="code_crew.pkl",
#   inputs={
#     "question": 'How do I build a RAG chain in LCEL?',
#     "context": str(concatenated_content)
#   }
# )

## Create Flow State

In [84]:
from typing import List

class CodeGenState(BaseModel):
    """
    State for the code generation flow
    """
    error: str = ""
    question: str = ""
    messages: List = []
    generation: str = ""
    iterations: int = 0
    max_iterations: int = 3

# Creating the Code Flow

In [88]:
# Importing CrewAI Flow related components
from crewai.flow.flow import Flow, listen, start, router

class CodeGenFlow(Flow[CodeGenState]):
  def check_code(self):
    print("---CHECKING CODE---")

    code_solution = self.state.generation
    imports = code_solution.imports
    code = code_solution.code

    try:
      exec(imports)
    except Exception as e:
      print("---CODE IMPORT CHECK: FAILED---")
      self.state.error = str(e)
      return "code_failed"

    try:
      exec(imports + "\n" + code)
    except Exception as e:
      print("---CODE BLOCK CHECK: FAILED---")
      self.state.error = str(e)
      return "code_failed"

    print("---NO CODE TEST FAILURES---")
    return "success"

  def fix_code(self):
    if self.state.error != "":
      print("---FIXING CODE---")
      # Create task for fixing code
      code_fix_task = Task(
          description="""You are a coding assistant with expertise in LCEL, LangChain expression language.
          Here is a full set of LCEL documentation:
          -------
          {context}
          -------

          The previous code attempt failed with the following error:
          {error}

          Your coding task:
          {question}

          Previous code attempt:
          {explanation}
          {imports}
          {code}

          Answer with a description of the code solution, followed by the imports, and finally the functioning code block.
          Ensure all imports are correct and the code is executable.""",
          expected_output= "A working code solution to the problem",
          agent=coding_assistant,
          output_pydantic=CodeSolution
      )

      # Create crew for fixing code
      fix_crew = Crew(
          agents=[coding_assistant],
          tasks=[code_fix_task]
      )

      # Execute fix
      result = fix_crew.kickoff(
          inputs={
              "error": self.state.error,
              "question": self.state.question,
              "explanation": self.state.generation.prefix,
              "imports": self.state.generation.imports,
              "code": self.state.generation.code,
              "context": concatenated_content
          }
      )
      self.state.generation = result.pydantic
      self.state.error = ""

  @start()
  def generate_code(self):
    print("---GENERATING CODE SOLUTION---")
    result = code_crew.kickoff(
      inputs={
        "question": self.state.question,
        "context": concatenated_content
      }
    )
    self.state.generation = result.pydantic
    self.state.error = ""

  @router(generate_code)
  def run_check(self):
    result = self.check_code()
    if result != "success":
      return "fix_code"

  @listen('fix_code')
  def run_fix(self):
    self.fix_code()

  @router(run_fix)
  def re_run_check(self):
    result = self.check_code()
    if result != "success":
      return "refix_code"

  @listen('refix_code')
  def re_run_fix(self):
    self.fix_code()

  @listen(re_run_fix)
  def re_re_run_check(self):
    self.check_code()



In [89]:
code_flow = CodeGenFlow()
code_flow.kickoff(inputs={"question": 'How do I build a RAG chain in LCEL?'})
code_flow.state.generation

---GENERATING CODE SOLUTION---


2024-12-16 11:42:24,147 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---FIXING CODE---


2024-12-16 11:42:38,562 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---FIXING CODE---
---CHECKING CODE---
Retrieval Augmented Generation (RAG) is a machine learning approach that combines retrieval-based methods with generative models to enhance the quality and relevance of generated text. In RAG, a system first retrieves relevant documents or information from a knowledge base based on the input query. This retrieved information is then used to inform and guide the generation of a response or text output. 

The process typically involves two main components: a retriever, which searches for and selects relevant documents, and a generator, which creates text based on both the input query and the retrieved documents. This approach allows the model to produce more accurate and contextually appropriate responses by leveraging external knowledge, making it particularly useful for tasks where factual accuracy and detail are important, such as question answering or content generation.
---NO CODE TEST FAILURES-

CodeSolution(prefix="To build a Retrieval Augmented Generation (RAG) chain in LCEL, you need to use existing runnable components like a valid retriever, a language model, and an output parser. Here, I'll correct the example to ensure that we have working components by simulating a simple retriever and question source within the code.", imports='from langchain_openai import ChatOpenAI\nfrom langchain_core.output_parsers import StrOutputParser\nfrom langchain_core.prompts import ChatPromptTemplate\nfrom langchain_core.runnables import RunnableParallel\nimport getpass\nimport os', code="class MockRetriever:\n    def __call__(self, args):\n        # Dummy implementation, replace with your actual retriever logic\n        return 'This is your retrieved context.'\n\nclass MockQuestionSource:\n    def __call__(self, args):\n        # Dummy implementation, use your actual question source logic\n        return args['question']\n\n# Assuming this block handles API key setup\nif not os.environ.get

# Evaluations

We will check for imports, code execution and overall compare with the correct solution.

In [90]:
def check_import(solution) -> dict:
    imports = solution.imports
    try:
        exec(imports)
        return {"key": "import_check", "score": 1}
    except Exception:
        return {"key": "import_check", "score": 0}


def check_execution(solution) -> dict:
    imports = solution.imports
    code = solution.code
    try:
        exec(imports + "\n" + code)
        return {"key": "code_execution_check", "score": 1}
    except Exception:
        return {"key": "code_execution_check", "score": 0}

In [91]:
import pandas as pd

# Load the evaluation data
df = pd.read_csv("eval.csv")

# Store evaluation results
results = []

for _, row in df.iterrows():
    question = row["question"]
    # Run the workflow for each question
    solution = app.invoke({"messages": [("user", question)], "iterations": 0, "error": ""})

    # Run evaluations
    import_check = check_import(solution["generation"])
    execution_check = check_execution(solution["generation"])

    # Store results
    result = {
        "question": question,
        "import_check": import_check["score"],
        "execution_check": execution_check["score"]
    }
    results.append(result)

# Convert results to dataframe
lg_df = pd.DataFrame(results)
print("\nEvaluation Results:")
print(lg_df)


---GENERATING CODE SOLUTION---
---CHECKING CODE---
content='Why do bears have hairy coats?\n\nBecause they look silly in sweaters!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 13, 'total_tokens': 27, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None} id='run-7ec4c116-9ed0-4780-be4c-7658ae35bf85-0' usage_metadata={'input_tokens': 13, 'output_tokens': 14, 'total_tokens': 27, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
---NO CODE TEST FAILURES---
---DECISION: FINISH---
content='Why do bears have hairy coats?\n\nBecause they look silly in sweaters!' additional_kwargs={'refusal': None} re

In [92]:
lg_df

,question,import_check,execution_check
0,How can I use a prompt and model to create a c...,1,1
1,How can I add memory to an arbitrary chain usi...,1,0
2,I've defined a LCEL runnable chain = prompt | ...,1,1
3,"I have a LCEL runnable, chain, and am passing ...",1,1
4,I am passing text key 'foo' to my prompt and w...,1,0
5,My LCEL map contains the key 'question'. What ...,1,1
6,I'm invoking a LCEL chain with a map that cont...,1,1
7,I’m passing {'a':1} and want to create an out...,1,1
8,How can I make the output of my LCEL chain a s...,1,1
9,How can I apply a custom function to one of th...,1,0


In [93]:
# Load the evaluation data
df = pd.read_csv("eval.csv")

# Store evaluation results
results = []

for _, row in df.iterrows():
    question = row["question"]
    # Run the workflow for each question
    code_flow = CodeGenFlow()
    code_flow.kickoff(inputs={"question": question})

    # Run evaluations
    import_check = check_import(code_flow.state.generation)
    execution_check = check_execution(code_flow.state.generation)

    # Store results
    result = {
        "question": question,
        "import_check": import_check["score"],
        "execution_check": execution_check["score"]
    }
    results.append(result)

# Convert results to dataframe
ca_df = pd.DataFrame(results)
print("\nEvaluation Results:")
print(ca_df)

---GENERATING CODE SOLUTION---


---CHECKING CODE---
content="I'm sorry, but I don't have real-time weather information. To get the current weather in New York, I recommend checking a reliable weather website or app for the most accurate and up-to-date information." additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 17, 'total_tokens': 57, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None} id='run-9bb0dfdc-5d7e-4378-aeb6-5edb6ac60af2-0' usage_metadata={'input_tokens': 17, 'output_tokens': 40, 'total_tokens': 57, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
---NO CODE TEST FAILURES---
content="I'm sorry, but I can't provide 

2024-12-16 11:48:37,955 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---FIXING CODE---


2024-12-16 11:48:46,033 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


...

---CODE BLOCK CHECK: FAILED---
---FIXING CODE---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---
---GENERATING CODE SOLUTION---
---CHECKING CODE---


2024-12-16 11:53:04,414 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


---CODE BLOCK CHECK: FAILED---
---FIXING CODE---
---CHECKING CODE---


2024-12-16 11:53:12,588 - 8071896128 - __init__.py-__init__:521 - WARNING: Overriding of current TracerProvider is not allowed


---CODE BLOCK CHECK: FAILED---
---FIXING CODE---
---CHECKING CODE---
---CODE BLOCK CHECK: FAILED---

Evaluation Results:
                                             question  import_check  \
0   How can I use a prompt and model to create a c...             1   
1   How can I add memory to an arbitrary chain usi...             1   
2   I've defined a LCEL runnable chain = prompt | ...             1   
3   I have a LCEL runnable, chain, and am passing ...             1   
4   I am passing text key 'foo' to my prompt and w...             1   
5   My LCEL map contains the key 'question'. What ...             1   
6   I'm invoking a LCEL chain with a map that cont...             1   
7   I’m passing {'a':1}  and want to create an out...             1   
8   How can I make the output of my LCEL chain a s...             1   
9   How can I apply a custom function to one of th...             1   
10  With a RAG chain in LCEL, why are documents re...             1   
11  I am passing a map with

In [94]:
ca_df

,question,import_check,execution_check
0,How can I use a prompt and model to create a c...,1,1
1,How can I add memory to an arbitrary chain usi...,1,0
2,I've defined a LCEL runnable chain = prompt | ...,1,1
3,"I have a LCEL runnable, chain, and am passing ...",1,0
4,I am passing text key 'foo' to my prompt and w...,1,0
5,My LCEL map contains the key 'question'. What ...,1,1
6,I'm invoking a LCEL chain with a map that cont...,1,1
7,I’m passing {'a':1} and want to create an out...,1,1
8,How can I make the output of my LCEL chain a s...,1,1
9,How can I apply a custom function to one of th...,1,1


In [95]:
# Compare evaluation metrics between LangGraph and CrewAI approaches
comparison_df = pd.DataFrame({
    'Metric': ['Import Check Pass Rate', 'Execution Check Pass Rate'],
    'LangGraph': [
        lg_df['import_check'].mean() * 100,
        lg_df['execution_check'].mean() * 100
    ],
    'CrewAI': [
        ca_df['import_check'].mean() * 100,
        ca_df['execution_check'].mean() * 100
    ]
})

# Format percentages to 2 decimal places
comparison_df['LangGraph'] = comparison_df['LangGraph'].round(2)
comparison_df['CrewAI'] = comparison_df['CrewAI'].round(2)

print("\nComparison of Evaluation Results (%):")
print(comparison_df)


Comparison of Evaluation Results (%):
                      Metric  LangGraph  CrewAI
0     Import Check Pass Rate       95.0   100.0
1  Execution Check Pass Rate       65.0    70.0
